In [3]:
%pip install lenskit

In [ ]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn
from lenskit import topn

In [4]:
import pandas as pd

In [9]:
ratings = pd.read_parquet(f'hdfs:/user/yj2369/train_combined_small_set.parquet')


FileNotFoundError: [Errno 2] No such file or directory: 'hdfs:/user/yj2369/train_combined_small_set.parquet'